# Chapter 17


# Introduction to Tests for Categorical Data



## Introduction to Tests for Categorical Data

So far, we have been investigating tests for data with continious values. But, many data types are categorical where we must work with count statistics. Our goal is to find methods for determining if the difference in counts in two or more groups are significant or not.    

Count data are quite common in many areas of analysis. A very few of the great many possible examples include:    
- A marketing manager wishes to determine if the counts of purchases between people offered a discount and those buying a full price are significant.    
- A wildlife biologist must determine if the annual differences in the counts of a population of animals from year to year are significant.   
- An educator must determine if improvements in teaching methods have lead to a significant improvement in student graduation rates.    
- A trafic engineer needs to determine if a change in placement of highway signs has lead to a significnatly lower rate of collisions.    
- Determine if a group of emails contain a specific word with unexpectedly high or low frequency?

In this chapter we focus on two key points for the aforementioned problem:   
- Which distribution can we use to test the differences in counts? For these problems the Chi square or $\chi^2$ distribution is used. The basic properties of the $\chi^2$ distribution were introduced in Chapter 7.  
- Which hypothesis tests can we apply to these? We will examine both a parametric test and non-parametric, or permutation based, tests.   

To run this notebook, execute the code in the cell below to import the required packages.  

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.stats.power as ssp
import matplotlib.pyplot as plt
from scipy.stats import chisquare, chi2, fisher_exact
import seaborn as sns
import math
%matplotlib inline

## Pearson's Chi-Squared Test


One of the earliest tests for count data, $\chi^2$ test, was developed by Karl Pearson around 1900. Despite its age, this test is still used today. The $\chi^2$ test is based on a simple statistic, he sum of the differences of outcomes from expectations.     

To understand the foregoing statement, start by considering that a random draw from some population will be in categority $i$ (or group $i$) with probability, $p_i$. By the axioms of probability, $\sum_{i=1}^k p_i = 1$ for the $k$ groups. And for $n$ samples the expected number in each group is $n\ p_i = m_i$. We can succinctly express these relationships as follows:      

$$\sum_{i=1}^k m_i = n \sum_{i=1}^k p_i = n$$

We can now define a statistic which has a $\chi^2$ distribution for an infinite sample, $n \rightarrow \infty$:

$$\chi^2 \sim  \sum_{i=1}^k \frac{(x_i - m_i)^2}{m_i} = \sum_{i=1}^k \frac{x_i^2}{m_i} - n$$

As with any probability density function, confidence intervals and p-values can be computed for the $\chi^2$ distribution. These intervals are used to compute p-values to construct the $\chi^2$ test.       

The Pearson Chi-squared test (also written as $\chi^2$) has the following properties:

- $\chi^2$ is an unpaired test for counts in different categories.
- Tests if the difference in counts for different categories are significant.   
- Or, test if a sample has expected make up of counts for different categories.
- The categories must be mutually exclusive. For example, does the patient have cancer? (yes/no)
- Chi-squared can be used as a **goodness of fit** test. That is to test if a sample is representative of a population.

> **Karl Pearson**    
> Karl Pearson was the dean of late 19th and early 20th century mathematical statisticians. Pearson was a student of Frances Gaulton, the inventor of the regression method. Willam Sealy Gossett was one of Pearsons's students. This realationship is said to be the insparation for Gossett's pseudonym 'Student'. 


<img src="../images/Karl_Pearson_1912.jpg" alt="Drawing" style="width:275x; height:350px"/>
<center>Karl Pearson in 1912: A scary looking statistics professor!</center>

> In many ways Pearson's influence on the mathematical foundations of statistics is still with us more than a century latter. Many of Pearson's methods are used on a daily basis. Most unfortunately, Pearson was also a eugenicist and a racist. His misuse of statistics in this area has tarnished his legacy.

### Building a Chi-squared table

The traditional way to apply a Chi-squared test is to first create a Chi-squared table. While such tables are not built piece by piece in modern practice, an example will help you understand the principles involved.     

In this example we use the results of an A-B test with three possible outcomes. For example, this type of test might be applied to determine if a new web site drives more customer purchases. 

the code in the cell below builds a simple Chi-squared table. The columns in the data frame are:

- The actual occurrence of events.
- The expected probability of these events. This is the **distribution of the null hypothesis**.
- The expected occurrence of events given the expected probabilities under the null hypothesis. In this case, these probabilities are computed from a sample of the current web site. In other words, the expected null distribution of counts is the same as the current site.   
- The difference between the occurrence and the expected number of events under the null hypothesis.
- The square of the difference.
- The squared difference normalized by the expected number of occurrences. The sum of these figures in the Chi-squared statistic. 

Execute the code and examine the results. 

In [ ]:
ab_data = pd.DataFrame({'Action':['Leave Page', 'Continue Purchase', 'Add More to Purchase'],
                       'occurrence':[55,43,22],
                       'expected_per':[0.6,0.3,0.1]})
sum_occurrence = ab_data.loc[:, 'occurrence'].sum()
ab_data.loc[:, 'expected_occurance'] = sum_occurrence * ab_data.loc[:, 'expected_per']
ab_data.loc[:, 'diff'] = ab_data.loc[:, 'occurrence'] - ab_data.loc[:, 'expected_occurance'] 
ab_data.loc[:, 'sqr_diff'] = ab_data.loc[:, 'diff'].apply(lambda x: x**2)
ab_data.loc[:, 'diff_expect'] = ab_data.loc[:, 'sqr_diff'].div(ab_data.loc[:, 'expected_occurance'], axis='index')
ab_data = pd.concat([ab_data,
                      pd.DataFrame({'Action': 'Totals',
                      'occurrence': sum_occurrence,
                       'expected_per':  [np.nan],
                      'expected_occurance': [np.nan],
                      'diff': [np.nan],
                      'sqr_diff': [np.nan],
                      'diff_expect': ab_data.loc[:, 'diff_expect'].sum()})], 
                        ignore_index =  True)
ab_data = ab_data[['Action', 'occurrence', 'expected_per', 'expected_occurance', 'diff', 'sqr_diff', 'diff_expect']]
ab_data                                       

This Chi-squared table is interpreted as follows:

- The $\chi$-squared test statistic is 13.708, which is computed as the sum of the squared differences normalized by the expected occurrences.
- The $\chi$-squared distribution has (3 Outcomes - 1) = 2 degrees of freedom. Degree of freedom is the number of outcome options (3) minus 1.

> **Exercise 17-1:** To interpret the results of the foregoing calculation you need to compute the p-value given the $\chi$-squared test statistic and the degrees of freedom. Use the [scipy.stata.chi2.cdf](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html) method is used to compute the p-value. 

In [ ]:
## Put your code below


> Given the p-value you have computed, how likely is it that the differences in counts between these categories arrise from random variation alone? 

> **Answer:**     

### Computing the Chi Squared Statistic

In the foregoing example we computed the Chi-squared statistic and p-value directly. In general, this is a somewhat cumbersome approach. Instead, we can use the [scipy.stats.chisquare](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html) function to compute these statistics. 

> **Exercise 17-2:** Use the `chisquare` function to compute the $\chi^2$ statistic and the p-value. Display these two values.    

In [ ]:
## Put your code below




> Does the value of the $\chi^2$ statistic and p-value agree with the values computed with the manually constructed $\chi^2$ table?  

> **Answer:**   

### Power of the test

Finally, we should check the power of our test. Recall from Chapter 13 that the **power of a test** is formally defined as:

$$power = P(reject\ H_0| when\ H_a\ is\ true)$$

In pain language, the **power is the probability of getting a positive result when the null hypothesis is not true**. Conversely, a test with **insufficient power will not detect a real effect**. Clearly, we want the most powerful test we can find for the situation. A powerful test gives a better chance of detecting an effect.  

Execute the code in the cell below and examine the results. In this case, we will use the [statsmodels.stats.power.GofChisquarePower().solve_power()](https://www.statsmodels.org/devel/generated/statsmodels.stats.power.GofChisquarePower.power.html#statsmodels.stats.power.GofChisquarePower.power) function to compute power vs. effect size at several numbers of observations. Execute this code to determine if this test has reasonable power.

In [ ]:
def plot_power(x, y, xlabel, title, label=None,  ax=None, color='red'):  
    if ax == None: _, ax = plt.subplots(figsize=(6,6))
    ax.plot(x, y, color=color, linewidth=2, label=label)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel('Power')
    if(label != None): plt.legend()

diffs = np.arange(start = 0.1, stop = 1.0, step = 0.01) 
_, ax = plt.subplots(figsize=(6,6))
for n_obs, color in zip([15, 30, 60, 120], ['red', 'gray', 'blue', 'orange']):
    powers = ssp.GofChisquarePower().power(effect_size = diffs, nobs=n_obs, n_bins = 3, alpha=0.05)
    plot_power(diffs, powers, xlabel = 'Difference', title = 'Power vs. difference', 
                      ax=ax, color=color, label='N_obs = ' + str(n_obs)) 

The power of the $\chi^2$ test changes considerably with the number of observations. It is apparent that for smaller effect sizes a large number of observations are required to get reasonable test power of 0.8 or 0.9.    

> **Exercise 17-3:**  The other way to view test power is by the sample size for various effect sizes. In the cell below create and execute the code to display a plot of test power vs. sample size for effect sizes, $[0.2, 0.3, 0.45]$. Use a sample range of 10 to 300 in steps of 5.       

In [ ]:
## Put your code below







> Examine your results. At what sample size is the power of the test approximately 0.8 or the various effect sizes, and what does this tell you about the relationship between effect size and required sample size?   

> **Answer:**     

## Fisher's Exact Test

Person's derivation of the $\chi^2$ test uses an asymptomatic expansion, which is strictly valid for $n \rightarrow \infty$. For larger samples, assumption of the the $\chi^2$ distribution is reasonably accurate. The assumptions behind the Chi-squared statistic breakdown when the sample size is small (e.g. number of occurrences $\le 10$). In this case you can use Fisher's Exact Test or a bootstrap test. In practice Fisher's exact test is rarely used, but it is interesting  to think about it anyway as an example of a **permutation test**. 

According to the story, in 1911 Ronald Fisher worked in the same institution with a talented aquatic botanist, Dr. Muriel Bristol, who was quite particular about how her tea was served. Dr. Bristol told Fisher that she could tell the difference between cups of tea where the milk had been poured into the cup before or after the tea was poured. 

<img src="../images/Bristol.png" alt="Drawing" style="width:450px; height:300px"/>
<center>Dr Muriel Bristol, noted aquatic botanist, and quite particular about her tea.

Fisher, was a bit skeptical. He challenged Dr. Bristol to a test. In this test, Fisher prepared eight cups of tea. Four of the cups of where prepared in Dr. Bristol's preferred manner and the other four the other way. The tea was prepare out of sight of Dr. Bristol. However, she knew that there were four cups prepared each way. The order of presentation of each cup of tea was randomized. Fisher served the cups of tea to Dr. Bristol and asked her how the tea had been poured. In every case, she was correct!

Fisher devised a **permutation test** to determine the likelihood that Dr Bristol could have simply guessed the correct outcome. He devised the following permutation table for the chances of success:

<img src="../images/tea.png" alt="Drawing" style="width:700px; height:300px"/>

The possible permutations of the ways that Dr Bristol could have correctly (success) or incorrectly (failure) identified the way each cup of tea was prepared is 8 choose 4:

$$\binom{8}{4} = \frac{8!}{4!(8-4)!} = 70$$

So, the chance that the Dr Bristol could purely guess (by random chance) the outcome is only 1 in 70 or about 1.4%. 

### Example of Fisher's Exact Test

As has already been stated, Fisher's exact test is a **small sample test**. Such tests are of limited importance in the 21st Century. There are still some cases where we must work with small samples. An example is with scientific data where collection of more data is expensive or simply infeasible.     

Let's try an example of Fisher's exact test. Take a simple 2X2 matrix of counts. The counts are number of success and failures for two samples (two sets of Bernoulli trails), on the number of sharks and whales observed in two oceans over some period of time. The null distribution in this cases is that the proportion of sharks to whales is constant between the two oceans. This is a case where the observations are inherently limited and Fisher's exact test can be used.   

The code in the cell below using the [scipy.stats.fisher_exact](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.fisher_exact.html) function to compute a Fisher exact test. Run the code and examine the results.  

In [ ]:
mat_test = pd.DataFrame(np.array([[8,2],[1,5]]),
                       index = ['wales', 'sharks'],
                       columns = ['Atlantic', 'Indian'])
print(mat_test)
oddsratio, pvalue = fisher_exact(mat_test)
print('\nOddsratio = ' + str(oddsratio))
print('P-value = ' + str(pvalue))

In this case, we can reject the null hypothesis. Evidently the difference of counts of wales and sharks in the two oceans from this experiment is unlikely to arise from the null distribution. 

## Example: Contingency Tables and the $\chi^2$ Test  

To demonstrate the methods described here, we will use a running example. A company wishes to change the alloy used for the production of a critical componenet. Using the new alloy will resuling in a lower cost and lighter part. However, there is concern that that the new alloy may lead to a higher defect rate, which would mitigate the potential cost advantage. To determine if this concern is significant and engineer tests 30 samples of the part made from the old material, 'Material 0' and the new material 'Material 1'. The goal is to use $\chi^2$ tests to determine if the defect rate is significantly higher for the new material.  

As a frist step, the engineer must determine how large of a sample must be tested, by calculating the power of the test. The economics of the manufacturing process require that the failure rate of the new material be no more than 20% higher than the old material.            

> **Exercise 17-4:** You will now investigate the power of the a $\chi^2$ test to detect an effect size of 0.2. To do so, create and execute code to do the following:    
> 1. Use sample size rage of 10 to 300 with a step size of 10.     
> 2. Use the [statsmodels.stats.power.GofChisquarePower().solve_power()](https://www.statsmodels.org/devel/generated/statsmodels.stats.power.GofChisquarePower.power.html#statsmodels.stats.power.GofChisquarePower.power) function with effect size of 0.2 and number of bins of 2, since there are two materials being compared. Use a significance level of 0.05.       
> 3. Display your results using the `plot_power` function provided.   

In [ ]:
## Put your code below 




> Examine the plot. Approximately, how large a sample size is required to reach a minimum detection probability of 0.8?  

> **Answer:**    

With the sample size determined, the engineer randomly samples and tests the requisite number of parts. One half from Material 0 and one half from Material 1.    

Execute the code in the cell below to load and view results of the engineer's tests on the two materials. A positive test in this case is has a positively identified manufacturing defect in the part.      

In [ ]:
MaterialTest = pd.read_csv('../data/MaterialTest.csv')
MaterialTest

### Contingency tables

As a first exploratory step in this anaysis, a [**contingency table**](https://en.wikipedia.org/wiki/Contingency_table) is constructed. A basic contingency table is a **cross tabulation** or cross-tab of counts for a pair of binary variables. The sums of counts for rows and columns are displayed on the margins. The total sum of all counts is show in the lower right corner.     

The code in the cell below computes constructs a continegency table in two steps:      
1. The sums of the counts are aggregated by the values of the two variables into a $2 \times 2$ table.    
2. The row, column and total sums are added to the margins of the table.     

Execute the code in the cell below and examine the result.  

In [ ]:
def aggregate_table(df):
    ## Create an empty data frame
    MaterialTable = pd.DataFrame(np.array([0]*4).reshape(2,2), columns=['Pass','Fail'], 
                             index=['Material 0', 'Material 1'])
    ## Fill in the contingency table
    for i,mat in enumerate(MaterialTable.index):  
        MaterialTable.loc[mat,'Fail'] = sum(df[df.Material == i]['Test Result'])
        MaterialTable.loc[mat,'Pass'] = len(df[df.Material == i]) - MaterialTable.loc[mat,'Fail']
    return MaterialTable


def build_contingency_table(MaterialTest):
    ## Create an empty data frame
    MaterialTable = pd.DataFrame(np.array([0]*9).reshape(3,3), columns=['Pass','Fail', 'Total'], 
                             index=['Material 0', 'Material 1', 'Total'])
    ## Fill in the contingency table
    MaterialTable.loc[['Material 0', 'Material 1'],['Pass','Fail']] = aggregate_table(MaterialTest)
    MaterialTable.loc[['Material 0', 'Material 1'],'Total'] = np.sum(MaterialTable.iloc[:2,:2],axis=1)
    MaterialTable.loc['Total',:] = np.sum(MaterialTable,axis=0)
    return MaterialTable

MaterialTable = build_contingency_table(MaterialTest)
MaterialTable

At first glance it appears that the new material does lead to a higher defect rate. Be one must be cautious, since this difference may just arrise from sampling alone and may not be significant. We will explore the significance with the $\chi^2$ test. 

### Summary statistics for contingency tables

To start our investigation of these data we will compute and interpret some summary statistics. Specifically, we will use the [**relative risk ratio](https://en.wikipedia.org/wiki/Relative_risk) and [**odds ratio**](https://en.wikipedia.org/wiki/Odds_ratio).       

**Relative Risk Ratio**     

The relative risk ratio is simply the ratio of the emperical probability of an event (failed test in this case) for a test or **treatment group** to the emperical probability of an event in the **control group**. Here, the control group is the original material, Material 0, and the treatment group is the new material to be tested, Material 1. For a contingency table, $C$, the risk ratio is then expressed:     

$$Risk\ ratio = \frac{c_{1,1} / (c_{1,0} + c_{1,1})}{c_{0,1} / (c_{0,0} + c_{0,1})} = \frac{c_{1,1} * (c_{0,0} + c_{0,1}) }{c_{0,1} * (c_{1,0} + c_{1,1})}$$

The standard error of the log of the risk ratio can be computed as:    

$$SE \big( log(risk\ ratio) \big) = \sqrt{\frac{c_{0,1}}{c_{0,0} * (c_{0,0} + c_{0,1})} + \frac{c_{1,1}}{c_{1,0} * (c_{1,0} + c_{1,1})}}$$

The relative risk ratio (RR) is both fairly simple and intuitive. Some key properties are:    
1. An RR of 0 indicates there is no difference in risk.     
2. An RR greater than 1 means the risk of a failure in the control group is less than the treatment group.    
3. An RR less than 1 means the risk of a failure in the control group is more than the treatment group.  
4. The RR looses sensitivity where there is a high proportion of events in either the treatment or control group.    
5. The standard error of the RR tells you how confident you should be in the point estimate. The standard error decreases with increasing sample size.       

**Odds Ratio**    

The odds ratio is a summary statistic applied to contingency tables. The odds ratio and log odds ratio are the odds that one case has different counts from another case. For a $2 \times 2$ contindency table, $C$, the odds ratio is defined as:  

$$Odds\ ratio = \frac{c_{1,1} / c_{1,2}}{c_{2,1} / c_{2,2}} = \frac{c_{1,1} * c_{2,2}}{c_{2,1} * c_{1,2}}$$

The standard error of the log of the odds ration can be computed as:    

$$SE \big( log(odds\ ratio) \big) = \sqrt{\frac{1}{c_{0,0}} + \frac{1}{c_{0,1}} + \frac{1}{c_{1,0}} + \frac{1}{c_{1,1}}}$$

Interpretation of the odds ratio (OR) requires a bit more thought than the risk ratio. Some key properties of the OR are:      
1. An odds ratio of $1.0$ (or log odds ratio = 0)indicates even odds of failure for the control and treatment groups are identical.   
2. An OR greater than $1.0$ (or log odds ratio greater than zero) indicates that more events (failures in our example) occur in the treatment group than the control group.     
3. An OR ratio less than $1.0$ (or log odds ratio less than zero) indicates that fewer events (failures in our example) occur in the treatment group than the control group.   
4. The OR is symmetric in the sense that either group can be considered the control. As a result, **odds ratio does not imply causality**.  
5. The odds ratio converges to the risk ratio when the fraction of events is small.       
6. The OR retains sensitivity for large fractions of events when compared to the risk ratio.   
7. The standard error of the OR tells you how confident you should be in the point estimate. The standard error of the RR tells you how confident you should be in the point estimate.      

To display these summary statistics, along with their standard errors, execute the code in the cell below and examine the results.  

In [ ]:
## Compute and display odds ratio   
risk_ratio = ((MaterialTable.iloc[1,1] / MaterialTable.iloc[1,2])/(MaterialTable.iloc[0,1] / MaterialTable.iloc[0,2]))
risk_ratio_se = math.sqrt((MaterialTable.iloc[0,0] /  (MaterialTable.iloc[0,1] * MaterialTable.iloc[0,2])) + 
                          (MaterialTable.iloc[1,0] / (MaterialTable.iloc[1,1] * MaterialTable.iloc[1,2])))
odds_ratio = MaterialTable.iloc[0,0] * MaterialTable.iloc[1,1]  /(MaterialTable.iloc[0,1] * MaterialTable.iloc[1,0])
std_err = math.sqrt(np.sum(np.sum(np.divide(1.0,MaterialTable.iloc[:2,:2]))))
print('Risk ratio ={:7.2f} +/-{:5.2f}'.format(risk_ratio, math.exp(risk_ratio_se)))
print('Log risk ratio ={:5.2f} +/-{:5.2f}'.format(math.log(risk_ratio), risk_ratio_se))
print('Odds ratio ={:5.2f} +/-{:5.2f}'.format(odds_ratio, math.exp(std_err)))
print('Log odds ratio ={:5.2f} +/-{:5.2f}'.format(math.log(odds_ratio),  std_err))

> **Exercise 17-5:** The question now is what do these summary statistics tell us about the significance, or lack significance, for the differences in test failures between the two materials. From these statistics and the associated standard errors does it appear that the differences in failure rates for the two materials is significant?      

> **Answer:**      

### Applying the $\chi^2$ test     

The preliminary expolration of the contingency table indicates that the failure rates for the two materials may not be significnatly different. We can continue to formalize this analysis by using a $\chi^2$ test.     

> **Exercise 17-6:** In the cell below apply the [scipy.stats.chisquare](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html) function to the values in the contingency table. Make sure you pay attention to the order of the arguments, with the expected values being for Material 0, and the observed values being for Material 1. Display the values of the $\chi^2$ statistic and the p-value returned by the function.   

In [ ]:
## Put your code below




> Do these results support the initial inference that the difference in failure rates for the two materials are the same, or not, and why?       

> **Answer:**     

## Summary

We have covered lot of ground in this lesson. Specifically we have discussed:

- The $\chi$-squared test for count data. The null hypothesis is that there is no significant differences in the counts and the samples are from the same population. 
- Power of tests on counts. The power of a test is the probability of getting a positive result when the null hypothesis is not true. 
- Fisher's exact test is a permutation test suitable for small count samples. 

#### Copyright 2019, 2020, 2023, Stephen F Elston. All rights reserved.